# Imports and Environment

In [17]:
# Basic pandas and numpy
import pandas as pd
import numpy as np
 
# Basic visualization tools
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
 
# Pandas defaults
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500
 
# Make jupyter bigger
from IPython.core.display import display, HTML
display(HTML('<style>.container { width:100% !important; }</style>'))

In [18]:
# Importing dataset:
df2 = pd.read_csv('../Data/df2.csv')

In [19]:
# Subsetting for the variables I want
dani_df2 = df2[['Survived', 'Pclass', 'Name', 'Sex', 'Ticket', 'Embarked', 'Cabin']]

In [20]:
dani_df2 = df2

# Type of variables and nulls

In [21]:
dani_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Ticket      891 non-null object
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 76.6+ KB


Only embarked and Cabin have NaNs

# Analysis: Target Variable - Survived

In [22]:
dani_df2.Survived.value_counts(normalize=True)

0    0.616162
1    0.383838
Name: Survived, dtype: float64

**Variable is numeric and has no NaN**

Aprox. 61.61% Did not survive (0)

Aprox. 38.38% Did survive (1)

# Analysis: Rest of Categorical Variables

## Embarked

**Nominal variable**

This is the only categorical variable that has nulls, but only two. Let's have a look at them:

In [23]:
dani_df2.loc[dani_df2.Embarked.isna()]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


With manual data found at [Encyclopedia Titanica](https://www.encyclopedia-titanica.org/) we conclude that both women embarked in Southampton (S), so we manually change these entries

In [24]:
dani_df2.Embarked[dani_df2.Embarked.isna()] = 'S'

/home/eiroamd/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
dani_df2.Embarked.value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

## Name

**Nominal variable**

In [26]:
# .head() is written for length issues when uploading to git, but the full series has been explored.
dani_df2.Name.value_counts().head()

Nicholson, Mr. Arthur Ernest         1
Burke, Mr. Jeremiah                  1
Johannesen-Bratthammer, Mr. Bernt    1
Lang, Mr. Fang                       1
Lievens, Mr. Rene Aime               1
Name: Name, dtype: int64

## Cabin

**Nominal variable**

In [27]:
# .head() is written for length issues when uploading to git, but the full series has been explored.
dani_df2.Cabin.value_counts()

G6                 4
C23 C25 C27        4
B96 B98            4
D                  3
F2                 3
F33                3
E101               3
C22 C26            3
C124               2
C123               2
C65                2
B20                2
E25                2
F4                 2
D35                2
B5                 2
E33                2
B28                2
C126               2
B22                2
D20                2
E121               2
C92                2
C83                2
B35                2
C52                2
D26                2
C2                 2
F G73              2
E67                2
B77                2
C93                2
D17                2
E24                2
B18                2
B51 B53 B55        2
C78                2
B49                2
C125               2
C68                2
D36                2
B57 B59 B63 B66    2
B58 B60            2
D33                2
E8                 2
E44                2
C118               1
C110         

For the cabin variable, the important thing for classification matters is the letter, so we will strip the rest in the 'Feature transformation' user story, bearing in mind that there are a lot of missing values.

## Sex

**Nominal variable**

In [28]:
dani_df2.Sex.value_counts(normalize=True)

male      0.647587
female    0.352413
Name: Sex, dtype: float64

Only males and females. No nulls. 

This variable is quite clean.

## Ticket

**Nominal variable**

In [29]:
# .head() is written for length issues when uploading to git, but the full series has been explored.
dani_df2.Ticket.value_counts(normalize=False).head()

CA. 2343    7
1601        7
347082      7
3101295     6
347088      6
Name: Ticket, dtype: int64

There are repeated values. We believe they belong to the same families, but try to demonstrate it

In [30]:
dani_df2.loc[dani_df2.Ticket == '347077']

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
25,1,3,"Asplund, Mrs. Carl Oscar (Selma Augusta Emilia...",female,38.0,1,5,347077,31.3875,NaN,S
182,0,3,"Asplund, Master. Clarence Gustaf Hugo",male,9.0,4,2,347077,31.3875,NaN,S
233,1,3,"Asplund, Miss. Lillian Gertrud",female,5.0,4,2,347077,31.3875,NaN,S
261,1,3,"Asplund, Master. Edvin Rojj Felix",male,3.0,4,2,347077,31.3875,NaN,S


This column is quite messy, with a lot of values.
Some pattern can be infered for 1st class passengers:

In [31]:
dani_df2[dani_df2.Pclass == 1].tail()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
871,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S
872,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,B51 B53 B55,S
879,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C
887,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


* WEP (WEP - WE/P) 
* PC
* Only number

For 2nd class passengers:

In [32]:
dani_df2[dani_df2.Pclass == 2].tail()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
866,1,2,"Duran y More, Miss. Asuncion",female,27.0,1,0,SC/PARIS 2149,13.8583,NaN,C
874,1,2,"Abelson, Mrs. Samuel (Hannah Wizosky)",female,28.0,1,0,P/PP 3381,24.0000,NaN,C
880,1,2,"Shelley, Mrs. William (Imanita Parrish Hall)",female,25.0,0,1,230433,26.0000,NaN,S
883,0,2,"Banfield, Mr. Frederick James",male,28.0,0,0,C.A./SOTON 34068,10.5000,NaN,S
886,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S


* CA (multiple formats)
* SC (multiple formats)
* SO (multiple formats)
* Only number

For 3rd class passengers:

In [33]:
dani_df2[dani_df2.Pclass == 3].tail()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
882,0,3,"Dahlberg, Miss. Gerda Ulrika",female,22.0,0,0,7552,10.5167,NaN,S
884,0,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.0500,NaN,S
885,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,NaN,Q
888,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
890,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.7500,NaN,Q


* CA (multiple formats)
* SP (multiple formats)
* AC (multiple formats)
* SO (multiple formats)
* SOTON (multiple formats)
* STON (multiple formats)
* Only number

**We will manage these issues in the feature creation/transformation section, although there is only a clear pattern for 1st class passengers**

## Pclass

**Ordinal variable**

In [34]:
dani_df2.Pclass.value_counts(normalize=True)

3    0.551066
1    0.242424
2    0.206510
Name: Pclass, dtype: float64

There are no nulls. Clean values.


We will try to change the values later as, counterintuitively, 3 in this particular case is worse than 1

# Export 

In [35]:
dani_df2.to_csv('../Data/df3.csv', index=False)